## Setup
Install accompanying pinon library. It is better to install in dev mode in the conda environment with `pip install -e .` Jupyter server must restart to see changes to conda environment.
Auto reload the pinon module.  Other imports may be ignored for auto reload e.g.  `%aimport -simfin`. Note use of option 2 for auto reload and excluding other modules, rather than option 1 and including only reloadable modules.  This is because option 1 does not work correctly with aliased imports.

In [6]:
# !pip install -e ..

%load_ext autoreload
%autoreload 2
%aimport -pandas
%aimport -simfin
%aimport -os

%aimport pinon

%matplotlib inline

# imports
import pandas as pd
import os

import pinon as pn
import pinon.names as pn_cols
import simfin as sf
import simfin.names as sf_cols  # column name shortcuts


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
config = pn.Config('master', 'V', 10)

Config file found at: /Users/daved/Dev/pinon/config/master.xlsx
Simfin data directory: /Users/daved/Dev/pinon/simfin_data
There is an error in the supplied ticker symbol EEFT in Company List of sheet: V of config file: /Users/daved/Dev/pinon/config/master.xlsx. Company uses non stardard quarterly reporting date not currently supported. Company was dropped from the list.
There is an error in the supplied ticker symbol PAGS in Company List of sheet: V of config file: /Users/daved/Dev/pinon/config/master.xlsx. Company was not found and was dropped from the list.


In [52]:
tr = pn.TradingRatios(config)
pe = tr.calc_quarterly_pe('V')

Adding breaking report for ticker V for report date 2023-03-31 00:00:00


Various experiments below

In [ ]:
dp = pn.DailyPrices()
spr = dp.get_downsampled_share_price_ratios('V')
pr = dp.get_downsampled_prices('V')
p = dp.get_daily_prices('V')
ratios = dp.get_daily_share_price_ratios('V')
ratios.loc['2022-10-1':'2022-12-30', sf_cols.PE_QUARTERLY]

In [ ]:
fund = pn.Fundamentals()
dr = fund.get_quarterly_derived_ratios('V')

In [ ]:
comp = pd.DataFrame(index = pe.index, columns=['Closing Price', 'Quarterly EPS', 'SimFin Quarterly EPS', 'PE TTM', 'SimFin PE TTM', 'Qtr PE', 'SimFin Qtr PE'])
comp['Closing Price'] = pr.loc[:, sf_cols.CLOSE]
comp['Quarterly EPS'] = pe.loc[:, 'Quarterly EPS']
comp['SimFin Quarterly EPS'] = dr.loc[:, sf_cols.EPS_DILUTED]
comp['PE TTM'] = pe.loc[:, 'TTM PE Ratio']
comp['SimFin PE TTM'] = spr.loc[:, sf_cols.PE_TTM]
comp['Qtr PE'] = pe.loc[:, pn_cols.QTR_PE_RATIO]
comp['SimFin Qtr PE'] = spr.loc[:, sf_cols.PE_QUARTERLY]

In [ ]:
p = dp.get_daily_prices('V')
inc1 = fund.get_quarterly_income_statement('V')
re_inc1 = inc1.asfreq(freq='D', method='ffill')
pe_x = pd.DataFrame(columns=['QTR_EPS', 'QTR_PE'])
# pe = p.loc[:,sf_cols.CLOSE] / re_inc1.loc[:, sf_cols.NET_INCOME] / re_inc1.loc[:, sf_cols.SHARES_DILUTED]
pe_x['QTR_EPS'] = re_inc1.loc[:, sf_cols.NET_INCOME] / re_inc1.loc[:, sf_cols.SHARES_DILUTED]
pe_x['QTR_PE'] = p[sf_cols.CLOSE] / pe_x['QTR_EPS']
pe_x.loc['2022-07-01':'2022-09-30', 'QTR_PE'].mean()
# p.loc['2022-10-01':'2022-12-30', sf_cols.CLOSE].mean()